In [1]:
import sqlite3
import pandas as pd

In [2]:
# connecting to the local database
connect = sqlite3.connect('dinofunworld.db')

# loading tables to pandas data frames
checkins_df = pd.read_sql_query("SELECT * FROM checkin", connect)
attractions_df = pd.read_sql_query("SELECT * FROM attraction", connect)
sequences_df = pd.read_sql_query("SELECT * FROM sequences", connect)

# function to convert duration string to total seconds that also deals with the database date format
def duration_to_seconds(duration_str):
    if duration_str is None:
        return None
    try:
        duration_part = duration_str.split('@')[0]
        hours, minutes, seconds = map(int, duration_part.split(':'))
        return hours * 3600 + minutes * 60 + seconds
    except ValueError:
        return None

In [3]:
# Answer 1: most popular attraction
popular_attractions = checkins_df['attraction'].value_counts()
most_popular_attraction_id = popular_attractions.idxmax()
most_popular_attraction_checkins = popular_attractions.max()
most_popular_attraction = attractions_df[attractions_df['AttractionID'] == most_popular_attraction_id]
most_popular_attraction_name = most_popular_attraction['Name'].iloc[0]

print(f"Answer 1: The most popular attraction is {most_popular_attraction_name} with {most_popular_attraction_checkins} checkins.")

Answer 1: The most popular attraction is Atmosfear with 27747 checkins.


In [8]:
rides_attractions_df = attractions_df[attractions_df['Category'].str.contains('rides', case=False)]
merged_df = pd.merge(checkins_df, rides_attractions_df, left_on='attraction', right_on='AttractionID')
merged_df['duration_seconds'] = merged_df['duration'].apply(duration_to_seconds)
merged_df.dropna(subset=['duration_seconds'], inplace=True)
longest_ride_duration_rounded = round(merged_df['duration_seconds'].max())
longest_rides_duration_attraction = rides_attractions_df[rides_attractions_df['AttractionID'] == merged_df.loc[merged_df['duration_seconds'] == longest_ride_duration_rounded, 'attraction'].iloc[0]]
longest_rides_duration_attraction_name = longest_rides_duration_attraction['Name'].iloc[0]

print(f"Answer 2: The ride with the longest visit time is {longest_rides_duration_attraction_name} reaching {longest_ride_duration_rounded} seconds.")

Answer 2: The ride with the longest visit time is Atmosfear reaching 47751 seconds.


In [5]:
# question 3: the least visited fast food

fast_food_attractions = attractions_df[attractions_df['type'] == 'Fast Food']
fast_food_visitors = checkins_df[checkins_df['attraction'].isin(fast_food_attractions['AttractionID'])]
visitor_counts = fast_food_visitors['attraction'].value_counts()
least_popular_fast_food_id = visitor_counts.idxmin()
least_popular_fast_food_visitors = visitor_counts.min()
least_popular_fast_food = attractions_df[attractions_df['AttractionID'] == least_popular_fast_food_id]
least_popular_fast_food_name = least_popular_fast_food['Name'].iloc[0]

print(f"Answer 3: The fast food with the fewest visitors is {least_popular_fast_food_name} having {least_popular_fast_food_visitors} visitors.")

Answer 3: The fast food with the fewest visitors is Theresaur Food Stop having 1116 visitors.


In [9]:
# question 4: skyline

# function to check if the ride is dominant
def is_dominant(ride, other_rides):
    for other_ride in other_rides:
        better_counts = [
            ride['visits'] > other_ride['visits'],
            ride['visit_time'] > other_ride['visit_time']
        ]
        if not any(better_counts):
            return False
    return True

rides_attractions_df = attractions_df[attractions_df['Category'].str.contains('rides', case=False)]
rides_visits = checkins_df[checkins_df['attraction'].isin(rides_attractions_df['AttractionID'])]
rides_visits = rides_visits.copy()
rides_visits.loc[:, 'duration_seconds'] = rides_visits.loc[:, 'duration'].apply(duration_to_seconds)
visits_count = rides_visits['attraction'].value_counts()
average_visit_time = rides_visits.groupby('attraction')['duration_seconds'].mean()
rides_stats = pd.DataFrame({'visits': visits_count, 'visit_time': average_visit_time}).reset_index()
rides_attractions = attractions_df[attractions_df['Category'].str.contains('rides', case=False)]

skyline_rides = []
for _, ride in rides_stats.iterrows():
    other_rides = rides_stats.loc[rides_stats.index != ride.name]
    if is_dominant(ride, other_rides.to_dict('records')):
        skyline_rides.append(ride['attraction'])

print("Answer 4: Rides in the skyline:")
for ride_id in skyline_rides:
    ride_name = attractions_df.loc[attractions_df['AttractionID'] == ride_id, 'Name'].values[0]
    print(f"{ride_name}")  


Answer 4: Rides in the skyline:
TerrorSaur
Atmosfear
Ichthyoroberts Rapids
Flight of the Swingodon
